In [2]:
from pyspark.sql import SparkSession

# 创建 SparkSession
spark = SparkSession.builder.appName("test").getOrCreate()

# 检查 Spark 版本
print(spark.version)


3.5.5


In [4]:
# Read the Parquet file into a Spark DataFrame
df = spark.read.parquet("yellow_tripdata_2024-10.parquet")
# Repartition the DataFrame into 4 partitions
df_repartitioned = df.repartition(4)
# Save the DataFrame to a new Parquet file
df_repartitioned.write.parquet("output/yellow_tripdata_2024-10-repartitioned.parquet")


In [6]:
df.head()

Row(VendorID=2, tpep_pickup_datetime=datetime.datetime(2024, 10, 1, 0, 30, 44), tpep_dropoff_datetime=datetime.datetime(2024, 10, 1, 0, 48, 26), passenger_count=1, trip_distance=3.0, RatecodeID=1, store_and_fwd_flag='N', PULocationID=162, DOLocationID=246, payment_type=1, fare_amount=18.4, extra=1.0, mta_tax=0.5, tip_amount=1.5, tolls_amount=0.0, improvement_surcharge=1.0, total_amount=24.9, congestion_surcharge=2.5, Airport_fee=0.0)

In [5]:
import os

# Define the output directory where the Parquet files are saved
output_dir = "output/yellow_tripdata_2024-10-repartitioned.parquet"

# Get a list of all files with .parquet extension
parquet_files = [f for f in os.listdir(output_dir) if f.endswith(".parquet")]

# Calculate total size in bytes
total_size = sum(os.path.getsize(os.path.join(output_dir, f)) for f in parquet_files)

# Convert size to MB
total_size_MB = total_size / (1024 * 1024)

# Compute the average size per file
average_size_MB = total_size_MB / len(parquet_files) if parquet_files else 0

print(f"Total Size: {total_size_MB:.2f} MB")
print(f"Average Parquet File Size: {average_size_MB:.2f} MB")


Total Size: 89.58 MB
Average Parquet File Size: 22.39 MB


In [11]:
from pyspark.sql import functions as F

# 确保 tpep_pickup_datetime 和 tpep_dropoff_datetime 都是时间戳类型
df = df.withColumn("tpep_pickup_datetime", F.col("tpep_pickup_datetime").cast("timestamp")) \
       .withColumn("tpep_dropoff_datetime", F.col("tpep_dropoff_datetime").cast("timestamp"))

# 提取日期部分并添加为新列 'pickup_date' 和 'dropoff_date'
df = df.withColumn("pickup_date", F.to_date("tpep_pickup_datetime")) \
       .withColumn("dropoff_date", F.to_date("tpep_dropoff_datetime"))

# 重新分区数据，按日期分区
df_repartitioned = df.repartition(4, "pickup_date")

# 保存重新分区的数据为Parquet
df_repartitioned.write.partitionBy("pickup_date").mode("overwrite").parquet("pickup_date_partition_output/yellow_tripdata_2024-10-repartitioned.parquet")

# 过滤出pickup_date和dropoff_date都是10月15日的记录
oct_15th_data = df_repartitioned.filter(
    (F.col("pickup_date") == F.to_date(F.lit("2024-10-15"))) & 
    (F.col("dropoff_date") == F.to_date(F.lit("2024-10-15")))
)

# 统计10月15日的出租车行程数量
trip_count = oct_15th_data.count()

print(f"There were {trip_count} taxi trips on the 15th of October.")

[Stage 19:=============================>                            (2 + 2) / 4]

There were 127993 taxi trips on the 15th of October.


In [12]:
# Calculate the duration of each trip in hours
df_with_duration = df_repartitioned.withColumn(
    "trip_duration_hours",
    (F.col("tpep_dropoff_datetime").cast("long") - F.col("tpep_pickup_datetime").cast("long")) / 3600  # Convert from seconds to hours
)

# Find the longest trip duration
longest_trip_duration = df_with_duration.agg(F.max("trip_duration_hours")).collect()[0][0]

# Output the result
print(f"The longest trip duration is {longest_trip_duration} hours.")

[Stage 25:==============>                                           (1 + 3) / 4]

The longest trip duration is 162.61777777777777 hours.


In [17]:
# Step 1: Load the taxi_zone_lookup data
zone_df = spark.read.option("header", "true").csv("taxi_zone_lookup.csv")

# Create a temporary view for the zone lookup data
zone_df.createOrReplaceTempView("taxi_zone_lookup")

joined_df = df_repartitioned.join(zone_df, df_repartitioned.PULocationID == zone_df.LocationID, "inner")
joined_df.head()

Row(VendorID=2, tpep_pickup_datetime=datetime.datetime(2024, 10, 6, 6, 6, 27), tpep_dropoff_datetime=datetime.datetime(2024, 10, 6, 6, 12, 14), passenger_count=1, trip_distance=1.85, RatecodeID=1, store_and_fwd_flag='N', PULocationID=238, DOLocationID=43, payment_type=1, fare_amount=10.0, extra=0.0, mta_tax=0.5, tip_amount=2.8, tolls_amount=0.0, improvement_surcharge=1.0, total_amount=16.8, congestion_surcharge=2.5, Airport_fee=0.0, pickup_date=datetime.date(2024, 10, 6), dropoff_date=datetime.date(2024, 10, 6), LocationID='238', Borough='Manhattan', Zone='Upper West Side North', service_zone='Yellow Zone')

In [18]:
pickup_counts = joined_df.groupBy("Zone").count()
pickup_counts.head()

Row(Zone='Homecrest', count=263)

In [19]:
least_frequent_zone = pickup_counts.orderBy("count").limit(1)

In [20]:
least_frequent_zone.show()

[Stage 48:=============================>                            (2 + 2) / 4]

+--------------------+-----+
|                Zone|count|
+--------------------+-----+
|Governor's Island...|    1|
+--------------------+-----+

